In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("20240627").getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/27 00:11:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
winter_df = spark.read.csv("hdfs:///encore/tpss_bcycl_od_statnhm_202001.csv", 
               encoding='cp949', header=True)

In [6]:
winter_df.printSchema()

root
 |-- 기준_날짜: string (nullable = true)
 |-- 구분코드: string (nullable = true)
 |-- 기준_시간: string (nullable = true)
 |-- 시작_대여소ID: string (nullable = true)
 |-- 시작_대여소명: string (nullable = true)
 |-- 종료_대여소ID: string (nullable = true)
 |-- 종료_대여소명: string (nullable = true)
 |-- 전체건수: string (nullable = true)
 |-- 전체이용시간(분): string (nullable = true)
 |-- 전체이용거리(m): string (nullable = true)



In [7]:
winter_df.first()

Row(기준_날짜='20200101', 구분코드='0', 기준_시간='0000', 시작_대여소ID='ST-443', 시작_대여소명='성산2동_041_2', 종료_대여소ID='ST-82', 종료_대여소명='성산2동_041_3', 전체건수='1', 전체이용시간(분)='2', 전체이용거리(m)='450')

In [16]:
from pyspark.sql.types import StructType, IntegerType, StringType, FloatType
schema = StructType().\
      add('기준_날짜',  StringType(), True ).\
        add('구분코드',  StringType(), True ).\
        add('기준_시간',  StringType(), True ).\
        add('시작_대여소ID',  StringType(), True ).\
        add('시작_대여소명',  StringType(), True ).\
        add('종료_대여소ID',  StringType(), True ).\
        add('종료_대여소명',  StringType(), True ).\
        add('전체건수',  IntegerType(), True ).\
        add('전체이용시간(분)',  IntegerType(), True ).\
        add('전체이용거리(m)' , FloatType(), True )

winter_df = spark.read.format("csv").\
            option("header", True).\
            option("encoding", "cp949").\
            schema(schema).csv("hdfs:///encore/tpss_bcycl_od_statnhm_202001.csv")
summer_df = spark.read.format("csv").\
            option("header", True).\
            option("encoding", "cp949").\
            schema(schema).csv("hdfs:///encore/tpss_bcycl_od_statnhm_202007.csv")


In [18]:
winter_df= winter_df.withColumn("기준_날짜", fn.to_date(fn.col("기준_날짜"), "yyyyMMdd"))

In [35]:
summer_df= summer_df.withColumn("기준_날짜", fn.to_date(fn.col("기준_날짜"), "yyyyMMdd"))
summer_df = summer_df.withColumn("요일", fn.date_format(fn.col("기준_날짜"),"E"))

In [36]:
summer_df.first()

Row(기준_날짜=datetime.date(2020, 7, 1), 구분코드='1', 기준_시간='0000', 시작_대여소ID='ST-883', 시작_대여소명='조원동_034_1', 종료_대여소ID='ST-1265', 종료_대여소명='조원동_014_1', 전체건수=1, 전체이용시간(분)=7, 전체이용거리(m)=0.0, 요일='Wed')

In [27]:
winter_df = winter_df.withColumn("요일", fn.date_format(fn.col("기준_날짜"),"E"))

In [28]:
winter_df.first()

Row(기준_날짜=datetime.date(2020, 1, 1), 구분코드='0', 기준_시간='0000', 시작_대여소ID='ST-443', 시작_대여소명='성산2동_041_2', 종료_대여소ID='ST-82', 종료_대여소명='성산2동_041_3', 전체건수=1, 전체이용시간(분)=2, 전체이용거리(m)=450.0, 요일='Wed')

In [15]:
import pyspark.sql.functions as fn
summer_df.agg(*[
    fn.count(fn.when(fn.isnull(x), x)).\
             alias(x) for x in summer_df.columns
            ]).show()

+---------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+
|기준_날짜|구분코드|기준_시간|시작_대여소ID|시작_대여소명|종료_대여소ID|종료_대여소명|전체건수|전체이용시간(분)|전체이용거리(m)|
+---------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+
|        0|       0|        0|            0|            0|            0|            0|       0|               0|              0|
+---------+--------+---------+-------------+-------------+-------------+-------------+--------+----------------+---------------+



## Groupby

In [30]:
simpleData = [("James","Sales","NY",90000,34,10000),
    ("Michael","Sales","NY",86000,56,20000),
    ("Robert","Sales","CA",81000,30,23000),
    ("Maria","Finance","CA",90000,24,23000),
    ("Raman","Finance","CA",99000,40,24000),
    ("Scott","Finance","NY",83000,36,19000),
    ("Jen","Finance","NY",79000,53,15000),
    ("Jeff","Marketing","CA",80000,25,18000),
    ("Kumar","Marketing","NY",91000,50,21000)
  ]

In [31]:
# Create DataFrame
schema = ["employee_name","department","state","salary","age","bonus"]
df = spark.createDataFrame(data=simpleData, schema = schema)
df.show(truncate=False)
df.groupBy("department").count().show(truncate=False)

+-------------+----------+-----+------+---+-----+
|employee_name|department|state|salary|age|bonus|
+-------------+----------+-----+------+---+-----+
|James        |Sales     |NY   |90000 |34 |10000|
|Michael      |Sales     |NY   |86000 |56 |20000|
|Robert       |Sales     |CA   |81000 |30 |23000|
|Maria        |Finance   |CA   |90000 |24 |23000|
|Raman        |Finance   |CA   |99000 |40 |24000|
|Scott        |Finance   |NY   |83000 |36 |19000|
|Jen          |Finance   |NY   |79000 |53 |15000|
|Jeff         |Marketing |CA   |80000 |25 |18000|
|Kumar        |Marketing |NY   |91000 |50 |21000|
+-------------+----------+-----+------+---+-----+

+----------+-----+
|department|count|
+----------+-----+
|Sales     |3    |
|Finance   |4    |
|Marketing |2    |
+----------+-----+



In [33]:
winter_df.groupBy("요일").count().orderBy("count").show(truncate=False)

+----+------+
|요일|count |
+----+------+
|Sun |170925|
|Sat |184020|
|Mon |185929|
|Tue |197435|
|Wed |289054|
|Fri |305148|
|Thu |336492|
+----+------+



In [38]:
summer_df.groupBy("요일").count().orderBy("count", ascending=False).show(truncate=False)

+----+------+
|요일|count |
+----+------+
|Fri |914310|
|Thu |820097|
|Wed |781305|
|Sat |747857|
|Tue |682796|
|Sun |597415|
|Mon |466809|
+----+------+



In [ ]:
friday_df = summer_df.filter(fn.col("요일") == "Fri")
friday_df.select("기준_시간", "전체이용시간(분)").orderBy("전체이용시간(분)").show()
#friday_df = df.filter(col("요일") == "Fri").select("기준_시간", "전체이용시간(분)")